# LSTM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade tensorflow-gpu==2.0
!pip install streamlit
!pip install pyngrok
!pip install plotly
!pip install --upgrade nbformat
!pip install nltk
!pip install spacy # spaCy is an open-source software library for advanced natural language processing
!pip install WordCloud
!pip install gensim # Gensim is an open-source library for unsupervised topic modeling and natural language processing
!pip install jupyterthemes


import nltk
nltk.download('punkt')

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow-gpu==2.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.12.0)
ERROR: No matching distribution found for tensorflow-gpu==2.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
# # In case of any corpus are missing 
# download all-nltk
nltk.download('stopwords')
stop_words = stopwords.words("english")

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow import keras
import tensorflow as tf
import pickle
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_excel("/content/cleaned-data.xlsx")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1718 entries, 0 to 1717
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         1718 non-null   int64 
 1   review_text        1712 non-null   object
 2   verified_purchase  1718 non-null   bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 28.6+ KB


In [ ]:
df.head()

,Unnamed: 0,review_text,verified_purchase
0,0,get older know like suitabl bodi like dove giv...,True
1,1,three gigant marmit jar last probabl whole lif...,True
2,2,excel,True
3,3,great flavour top slow cook,True
4,4,say,False


In [ ]:
df.tail()

,Unnamed: 0,review_text,verified_purchase
1713,1713,amaz like origin cornetto vegan im realli impress,False
1714,1714,whole famili love bovril daili toast morn year...,False
1715,1715,content bottl love howev tri get squeezi bottl...,False
1716,1716,tast great cannot cant tell vegan,False
1717,1717,realli pleas vegan cornetto last close dairi v...,False


In [ ]:
df=df.dropna()

In [ ]:
X=df.drop('verified_purchase',axis=1)

In [ ]:
y=df['verified_purchase']

In [ ]:
X.shape

(1712, 2)

In [ ]:
y.shape

(1712,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

### Onehot Representation


In [ ]:
messages=X.copy()

In [ ]:
messages['review_text'][1]

'three gigant marmit jar last probabl whole life els would possibl wish order came time mention safe pack happi'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['review_text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
corpus

['get older know like suitabl bodi like dove give fresh wide awak feel dandruff flakey smelli pit',
 'three gigant marmit jar last probabl whole life el would possibl wish order came time mention safe pack happi',
 'excel',
 'great flavour top slow cook',
 'say',
 'excel item good men',
 'thank happi',
 'beauti',
 'alway buy big bottl',
 'alway heavili mistreat hair work wonder time time',
 'great valu',
 'one pictur get star regular one awesom peopl dri',
 'item describ excel qualiti rapid deliveri great irrit red blemish',
 'good bodi wash strong smell',
 'great',
 'worth',
 'rememb tri friend hou year ago ive month one ill buy',
 'great',
 'best antiperspir unless like go work snow drift arm',
 'love',
 'die eat one u die content man love u em',
 'love rang bodi lotion u greasi easi appli soak quickli leav feel super soft moisturi great valu money scent fresh light great go beauti essenti bathroom cupboard review collect part promot',
 'suffer dri sensit vaselin one make flare itch 

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4140,
  4068,
  2544,
  1499,
  4094,
  2683,
  1499,
  2810,
  705,
  575,
  2518,
  4257,
  4299,
  3187,
  729,
  10,
  2391],
 [3482,
  4007,
  1300,
  4325,
  1747,
  3285,
  2099,
  833,
  905,
  3576,
  3428,
  3043,
  248,
  3889,
  528,
  3475,
  2883,
  394,
  3460],
 [926],
 [4131, 1524, 160, 3884, 732],
 [197],
 [926, 2104, 1242, 169],
 [3379, 3460],
 [2052],
 [2439, 4814, 2055, 2805],
 [2439, 2712, 4275, 4872, 312, 4645, 528, 528],
 [4131, 4405],
 [4266, 2298, 4140, 4417, 1929, 4266, 1361, 2375, 245],
 [2104, 2679, 926, 1866, 1337, 4513, 4131, 222, 4106, 4440],
 [1242, 2683, 531, 3096, 3017],
 [4131],
 [4067],
 [2766, 944, 1083, 4328, 1668, 4665, 4854, 2297, 4266, 3052, 4814],
 [4131],
 [4331, 2898, 330, 1499, 3340, 312, 1188, 4262, 94],
 [222],
 [1101, 1359, 4266, 1383, 1101, 2113, 3279, 222, 1383, 4546],
 [222,
  3839,
  2683,
  3985,
  1383,
  4664,
  1789,
  4744,
  1799,
  1852,
  4451,
  4299,
  1961,
  4780,
  4922,
  4131,
  4405,
  1231,
  3962,
  575,
  2377,
 

### Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  729   10 2391]
 [   0 3482 4007 ... 2883  394 3460]
 [   0    0    0 ...    0    0  926]
 ...
 [4140 2364 2805 ... 4140 2113 2805]
 [   0    0    0 ... 2275 3444 3252]
 [   0    0    0 ... 2947 1931 3460]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0, 4140, 4068, 2544, 1499, 4094, 2683, 1499, 2810,
        705,  575, 2518, 4257, 4299, 3187,  729,   10, 2391], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(1712, (1712,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((1712, 20), (1712,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=64)

Epoch 1/100
18/18 [==============================] - 4s 73ms/step - loss: 0.6165 - accuracy: 0.7716 - val_loss: 0.4992 - val_accuracy: 0.7788
Epoch 2/100
18/18 [==============================] - 1s 36ms/step - loss: 0.4602 - accuracy: 0.7899 - val_loss: 0.4304 - val_accuracy: 0.8088
Epoch 3/100
18/18 [==============================] - 1s 30ms/step - loss: 0.4096 - accuracy: 0.8073 - val_loss: 0.4035 - val_accuracy: 0.8195
Epoch 4/100
18/18 [==============================] - 1s 29ms/step - loss: 0.3497 - accuracy: 0.8518 - val_loss: 0.3720 - val_accuracy: 0.8442
Epoch 5/100
18/18 [==============================] - 1s 30ms/step - loss: 0.2817 - accuracy: 0.8867 - val_loss: 0.4069 - val_accuracy: 0.8496
Epoch 6/100
18/18 [==============================] - 1s 30ms/step - loss: 0.2255 - accuracy: 0.9137 - val_loss: 0.4867 - val_accuracy: 0.8407
Epoch 7/100
18/18 [==============================] - 1s 29ms/step - loss: 0.1958 - accuracy: 0.9250 - val_loss: 0.4626 - val_accuracy: 0.8478
Epoch 

### Adding Dropout

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [ ]:
y_pred=  model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

18/18 [==============================] - 1s 8ms/step


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[194,  99],
       [ 72, 200]])

In [ ]:
#LSTM accuracy:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6973451327433628

In [ ]:
!mkdir -p saved_model
model.save('lstm_saved_model/my_model') 

In [ ]:

import tensorflow as tf
new_model = tf.keras.models.load_model('lstm_saved_model/my_model')

In [ ]:
model.save('lstm_model.h5') 

In [ ]:
with open("/content/lstm_tokenizer.pkl", "wb") as f:
 pickle.dump(tokenizer, f)